# PWV05 : Compare PWV  For differrent targets

**Goal :** PWV vs time in Spectrogram and Spectrum, and difference per observation

- author Sylvie Dagoret-Campagne
- creation date 2026-02-08 : version run2026_v01
- last update : 2026-02-10
- last update : 2026-02-19 : runs from Corentin Ravoux : generate images with name having the run tag
- affiliation : IJCLab
- Kernel @usdf **w_2026_02*
- Home emac : base (conda)
- laptop : conda_py313

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
# must install the mysitcom package by doing at top level "pip install --user -e . "
from mysitcom.auxtel.pwv import scatter_datetime
from mysitcom.auxtel.pwv import strip_datetime
from mysitcom.auxtel.pwv import bar_counts_by_night
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_filter
from mysitcom.auxtel.pwv import stripplot_target_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import summarize_dccd_chi2
from mysitcom.auxtel.pwv import plot_atmparam_vs_time, plot_atmparam_diff_vs_time
from mysitcom.auxtel.pwv import plot_atmparam_hist_per_filter, plot_atmparam_diff_hist_per_filter
from mysitcom.auxtel.pwv import plot_atmparam_vs_time_byfilter_bytarget,plot_atmparam_hist_stacked_bytarget

from mysitcom.auxtel.pwv import GetNightMidnightsDict,GetNightBoundariesDict

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV05"
prefix = "pwv05"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
#plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares

from pprint import pprint

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
from getCalspec import getCalspec
from getCalspec.getCalspec import getCalspecDataFrame

In [ ]:
# Remove to run faster the notebook
import ipywidgets as widgets
%matplotlib widget

In [ ]:
from PWV00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

In [ ]:
the_suptitle = tag

### Configuration

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `PWV00_parameters.py` 

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
inputfilename = atmfilename.split("/")[-1]

if "parquet" in inputfilename:
    df_spec = pd.read_parquet(atmfilename)
elif "npy" in inputfilename and "run_v" in version_run :
    specdata = np.load(atmfilename,allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
    df_spec["D_CCD [mm]"] = df_spec["D2CCD"]
    df_spec["PWV [mm]"] = df_spec["PWV [mm]_x"] 
    df_spec["PWV [mm]_rum"] = df_spec["PWV [mm]_y"] 
    df_spec["PWV [mm]_err"] = df_spec["PWV [mm]_err_x"] 
    df_spec["PWV [mm]_err_rum"] = df_spec["PWV [mm]_err_y"] 


    cols = [
    "PWV [mm]",
    "PWV [mm]_rum",
    "PWV [mm]_err",
    "PWV [mm]_err_rum",
    ]
    df_spec = df_spec.dropna(subset=cols)
else:
    specdata = np.load(atmfilename, allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
    
#else:
#    raise "bad path of filename {inputfilename}"
    

In [ ]:
FLAG_RENAME_SPECTROGRAM_VARIABLES = True

if FLAG_RENAME_SPECTROGRAM_VARIABLES and "run2026_v01" in version_run:
    df_spec.rename(
    {
    "chi2":"chi2_ram",
    "A1":"A1_ram",
    "A1_err": "A1_err_ram",
    "A2": "A2_ram",
    "A2_err": "A2_err_ram",
    "A3": "A3_ram",
    "A3_err": "A3_err_ram", 
    "VAOD": "VAOD_ram", 
    "VAOD_err": "VAOD_err_ram", 
    "angstrom_exp" : "angstrom_exp_ram", 
    "angstrom_exp_err" : "angstrom_exp_err_ram" , 
    "ozone [db]" :"ozone [db]_ram", 
    "ozone [db]_err": "ozone [db]_err_ram", 
    "PWV [mm]":  "PWV [mm]_ram",
    "PWV [mm]_err":"PWV [mm]_err_ram" , 
    "B": "B_ram" , 
    "B_err" : "B_err_ram", 
    "A_star": "A_star_ram" , 
    "A_star_err": "A_star_err_ram" , 
    "D_CCD [mm]" : "D_CCD [mm]_ram", 
    "D_CCD [mm]_err": "D_CCD [mm]_err_ram" 
    }
    ,axis=1,inplace = True)
elif FLAG_RENAME_SPECTROGRAM_VARIABLES and "run2026_v02" in version_run:
    df_spec["chi2_ram"] = df_spec["CHI2_FIT"]
    df_spec["chi2_rum"] = df_spec["CHI2_FIT"]

In [ ]:
print(" | ".join(df_spec.columns)) 

In [ ]:
#df_spec.dtypes.to_frame('Type de donnée')

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

## Select only empty and OG550 filters

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
if FLAG_PWVFILTERS: 
    df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTER_LIST) ]

### Check Filters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4),layout="constrained")

strip_datetime(
    df=df_spec,
    x="Time",
    y="FILTER",
    hue="FILTER",
    ax=ax,
    size=9,
)
ax.set_title(tag)
plt.show()


## Palette with more clear target seperation colors

In [ ]:
# Comptage et tri
target_counts = (
    df_spec['TARGET']
    .value_counts()
    .sort_values(ascending=False)
)
targets = target_counts.index.tolist()
counts = target_counts.values

In [ ]:
chosen_palette = "tab20"

if chosen_palette == "husl":
    palette = sns.color_palette("husl", n_colors=len(targets))
    #palette = sns.color_palette("husl", len(targets))[::-1]
    target_color_map = OrderedDict(zip(targets, palette))
elif chosen_palette == "hsv":
    base_palette = sns.color_palette("hsv", n_colors=len(targets))
    # réordonnancement pour maximiser contraste local
    order = np.arange(len(base_palette))
    order = np.roll(order, len(order)//2)
    palette = [base_palette[i] for i in order]
    target_color_map = OrderedDict(zip(targets, palette))
    #target_color_map = OrderedDict(zip(targets, palette[::-1]))
elif chosen_palette == "tab20":
    palette = sns.color_palette("tab20b", 20) + sns.color_palette("tab20c", 10)
    palette = palette[:len(targets)]
    target_color_map = OrderedDict(zip(targets, palette))
    #target_color_map = OrderedDict(zip(targets, palette[::-1]))
else:
    palette = sns.color_palette("viridis", n_colors=len(targets))
    #palette = sns.color_palette("viridis", n_colors=len(targets))[::-1]
    target_color_map = OrderedDict(zip(targets, palette)) 

# Colormap discrete
cmap = mpl.colors.ListedColormap(palette)
norm = mpl.colors.BoundaryNorm(boundaries=range(len(targets)+1),ncolors=len(targets))

In [ ]:
ordered_list_of_targets = list(target_color_map.keys())

In [ ]:
fig = plt.figure(figsize=(0.6*len(targets), 3),layout="constrained")

# axe très épais (0.15)
cax = fig.add_axes([0.05, 0.7, 0.9, 0.15])  
# [left, bottom, width, height]

cb = mpl.colorbar.ColorbarBase(
    cax,
    cmap=cmap,
    norm=norm,
    orientation='horizontal'
)

cb.set_ticks([i + 0.5 for i in range(len(targets))])
cb.set_ticklabels(targets)
cb.ax.tick_params(labelrotation=90)
cb.set_label("TARGET", labelpad=10)
cb.ax.tick_params(labelsize=10,length=6,width=1.5)

plt.suptitle(tag)
figfilename = f"{pathfigs}/{prefix}_{version_run}_palette_{chosen_palette}_targetnames{figtype}"
fig.savefig(figfilename)

#fig.show()


In [ ]:
fig, ax = plt.subplots(figsize=(6, 0.35*len(targets)),layout="constrained")

sns.barplot(
    x=counts,
    y=targets,
    palette=palette,
    ax=ax,
)

ax.set_xlabel("Number of Obs")
ax.set_ylabel("TARGET")
ax.set_title(f"TARGET observed ({tag})")
ax.tick_params(axis="y", labelsize=10)

#plt.tight_layout()

figfilename = f"{pathfigs}/{prefix}_{version_run}_baplottargets_palette_{chosen_palette}{figtype}"
plt.savefig(figfilename)
plt.show()

# Plot PWV vs time before 

### Calculate difference and ratio for some params

In [ ]:
denom = np.sqrt(df_spec["PWV [mm]_err_ram"]**2 + df_spec["PWV [mm]_err_rum"]**2)

df_spec["diff_PWV_norm"] = np.where(
    np.isfinite(denom) & (denom > 0),
    (df_spec["PWV [mm]_ram"] - df_spec["PWV [mm]_rum"]) / denom,
    np.nan
)

df_spec["diff_PWV"] =  (df_spec["PWV [mm]_ram"] - df_spec["PWV [mm]_rum"]) 
df_spec["diff_PWV_err"] = np.sqrt( (df_spec["PWV [mm]_err_ram"]**2 - df_spec["PWV [mm]_err_rum"]**2)) 

## Calculate midnights and night boundaries

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

In [ ]:
# get night boundaries
dn = GetNightBoundariesDict(df_spec)
# get midnights
dnidnights = GetNightMidnightsDict(df_spec)

In [ ]:
df_spec["FILTER"].unique()

### Plot PWV in spectrogram vs Time before quality cuts

In [ ]:
fig,ax = plot_atmparam_vs_time_byfilter_bytarget(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_ram",
    param_err_col = "PWV [mm]_err_ram",
    title_param = "PWV vs time (spectrogram no cut qual. cut)",
    target_color_map= target_color_map,
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plotvstime_pwvram_all-targets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()



### Plot PWV in spectrum vs Time

In [ ]:
fig,ax = plot_atmparam_vs_time_byfilter_bytarget(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_rum",
    param_err_col = "PWV [mm]_err_rum",
    title_param = "PWV vs time (spectrum no cut qual. cut)",
    target_color_map= target_color_map,
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)


plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plotvstime_pwvrum_all-targets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()


## Plot spectrogram - spectrum

In [ ]:
fig,ax = plot_atmparam_vs_time_byfilter_bytarget(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "diff_PWV",
    param_err_col = "diff_PWV_err",
    title_param = "$\Delta$ PWV vs time (ram-rum)",
    target_color_map= target_color_map,
    
    # seuils / bornes
    param_min_fig=-PWVMAX/5,
    param_max_fig=PWVMAX/5,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)



plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plotvstime_diffpwv_all-targets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()


## Histogram

In [ ]:
fig, ax = plot_atmparam_hist_stacked_bytarget(
    df_spec,
    param_col="diff_PWV",
    target_col="TARGET",
    target_color_map = target_color_map,
    bins=100,
    value_range = (-PWVMAX/15,PWVMAX/15),
)

plt.suptitle(the_suptitle)
plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plothistoliny_diffpwv_all-targets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, ax = plot_atmparam_hist_stacked_bytarget(
    df_spec,
    param_col="diff_PWV",
    target_col="TARGET",
    target_color_map = target_color_map,
    bins=100,
    value_range = (-PWVMAX/5,PWVMAX/5),
    ylogscale=True
)

plt.suptitle(the_suptitle)
plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plothistology_diffpwv_all-targets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, ax = plot_atmparam_hist_stacked_bytarget(
    df_spec,
    param_col="diff_PWV",
    target_col="TARGET",
    filter_value="empty",
    target_color_map = target_color_map,
    bins=100,
    value_range = (-PWVMAX/5,PWVMAX/5),
    ylogscale=True
)

plt.suptitle(the_suptitle)
plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plothistology_diffpwv_all-targets_filtempty-nocut"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
fig, ax = plot_atmparam_hist_stacked_bytarget(
    df_spec,
    param_col="diff_PWV",
    target_col="TARGET",
    filter_value="OG550_65mm_1",
    target_color_map = target_color_map,
    bins=100,
    value_range = (-PWVMAX/5,PWVMAX/5),
    ylogscale=True
)

plt.suptitle(the_suptitle)
plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plothistology_diffpwv_all-targets_filtog550-nocut"+figtype
plt.savefig(figname)
plt.show()

## Filter to select Gaia

In [ ]:
flag_calspec = OrderedDict()
for target_name in ordered_list_of_targets:
    flag_calspec[target_name] = getCalspec.is_calspec(target_name)

In [ ]:
df_calspec = pd.DataFrame.from_dict(flag_calspec, orient='index', columns=['is_calspec'])

In [ ]:
non_calspec_targets = df_calspec.index[~df_calspec["is_calspec"]]
non_calspec_targets

In [ ]:
# sélection dans df_spec
df_spec_non_calspec = df_spec[df_spec["TARGET"].isin(non_calspec_targets)]

In [ ]:
palette_non_calspec = []
ordered_list_of_non_calspec_targets = []
for idx,target in enumerate(ordered_list_of_targets):
    if not df_calspec.loc[target,"is_calspec"]:
        palette_non_calspec.append(palette[idx]) 
        ordered_list_of_non_calspec_targets.append(target)

In [ ]:
# Comptage et tri
target_counts = (
    df_spec_non_calspec['TARGET']
    .value_counts()
    .sort_values(ascending=False)
)
targets = target_counts.index.tolist()
counts = target_counts.values

In [ ]:
fig, ax = plt.subplots(figsize=(6, 0.3*len(targets)))

sns.barplot(
    x=counts,
    y=targets,
    palette=palette_non_calspec,
    ax=ax
)

ax.set_xlabel("Number of Obs")
ax.set_ylabel("TARGET")
ax.set_title("Gaia TARGET observed")

plt.suptitle(the_suptitle)

plt.tight_layout()
figfilename = f"{pathfigs}/{prefix}_{version_run}_baplotgaiatargets_palette_{chosen_palette}{figtype}"
plt.savefig(figfilename)

plt.show()

In [ ]:
df_spec_non_calspec.tail()

### PWV Spectrogram vs time for Gaia

In [ ]:
fig,ax = plot_atmparam_vs_time_byfilter_bytarget(
    df_spec_non_calspec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_ram",
    param_err_col = "PWV [mm]_err_ram",
    title_param = "PWV vs time for Gaia (spectrogram no cut qual. cut)",
    target_color_map= target_color_map,
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle = the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)
    

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plotvstime_pwvram_gaiatargets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()



### PWV Spectrum vs time for Gaia

In [ ]:
fig,ax = plot_atmparam_vs_time_byfilter_bytarget(
    df_spec_non_calspec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_rum",
    param_err_col = "PWV [mm]_err_rum",
    title_param = "PWV vs time for Gaia (spectrum no cut qual. cut)",
    target_color_map= target_color_map,
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle = the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)


plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plotvstime_pwvrum_gaiatargets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()

### PWV Spectrogram- Spectrum vs time for Gaia

In [ ]:
fig,ax = plot_atmparam_vs_time_byfilter_bytarget(
    df_spec_non_calspec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "diff_PWV",
    param_err_col = "diff_PWV_err",
    title_param = "$\Delta$ PWV vs time for Gaia (ram-rum)",
    target_color_map= target_color_map,
    
    # seuils / bornes
    param_min_fig=-PWVMAX/5,
    param_max_fig=PWVMAX/5,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle = the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)



plt.tight_layout()
figname =f"{pathfigs}/{prefix}_{version_run}_plotvstime_diffpwv_gaiatargets_allfilters-nocut"+figtype
plt.savefig(figname)
plt.show()